In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
pio.renderers.default='notebook'

#
# Used to implement the multi-dimensional counter we need in the performance class
from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)
print(keras.__version__)

2022-11-29 13:01:25.697465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 13:01:26.785620: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /apps/python/3.9-2022.05/lib:/apps/cuda/9.0.176/lib64:/apps/cuda/9.0.176/lib
2022-11-29 13:01:26.785655: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 13:01:30.234672: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libn

2.11.0
2.11.0


In [2]:
file = '41598_2020_70544_MOESM2_ESM.csv'

df = pd.read_csv(file, header=None, skiprows=2)

In [3]:
headers = df.iloc[0]
df  = pd.DataFrame(df.values[1:], columns=headers)

In [4]:
df.head(len(df))

,Player 42 Data:\nRound number:,Player 42 played:,AI 1 played:,AI 1 Score:,AI 1 Last 5 Rounds Total Score:,AI 2 played:,AI 2 Score:,AI 2 Last 5 Rounds Total Score:,AI 3 played:,AI 3 Score:,...,AI 8 played:,AI 8 Score:,AI 8 Last 5 Rounds Total Score:,AI 9 played:,AI 9 Score:,AI 9 Last 5 Rounds Total Score:,AI 10 played:,AI 10 Score:,AI 10 Last 5 Rounds Total Score:,Multi AI played:
0,1,P,P,0,0,P,0,0,R,-1,...,R,-1,0,P,0,0,P,0,0,P
1,2,R,P,1,0,R,0,0,S,-1,...,P,1,-1,P,1,0,P,1,0,R
2,3,R,P,1,1,P,1,0,S,-1,...,R,0,0,S,-1,1,P,1,1,P
3,4,S,P,-1,2,P,-1,1,S,0,...,S,0,0,R,1,0,S,0,2,P
4,5,P,P,0,1,S,1,0,R,-1,...,P,0,0,S,1,1,P,0,2,P
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296,R,P,1,2,S,-1,-3,R,0,...,R,0,-1,R,0,0,P,1,1,P
296,297,P,P,0,3,R,-1,-3,S,1,...,P,0,-1,P,0,0,S,1,1,P
297,298,S,R,1,2,R,1,-3,R,1,...,R,1,-1,S,0,0,S,0,1,R
298,299,R,S,-1,2,P,1,-2,P,1,...,S,-1,0,S,-1,1,S,-1,2,S


In [5]:
df_data = pd.DataFrame(columns=["Player_Choice", "CPU_Choice"])
df_data["Player_Choice"] = df["Player 42 played:"].replace(["R", "P", "S"],[0,1,2])
df_data["CPU_Choice"] = df["Multi AI played:"].replace(["R", "P", "S"],[0,1,2])
df_data.head(len(df_data))

,Player_Choice,CPU_Choice
0,1,1
1,0,0
2,0,1
3,2,1
4,1,1
...,...,...
295,0,1
296,1,1
297,2,0
298,0,2


In [6]:
import random

foo = df_data['Player_Choice'].tolist()
bar = df_data['CPU_Choice'].tolist()

train_data = []
train_labels = []

for i in range(250):
    x = random.randint(0,242)
    y_1 = []
    y_2 = []
    for j in range(x, x+8):
        if j < x+7:
            y_1.append(foo[j])
        else:
            y_2.append(bar[j])
    train_data.append(y_1)
    train_labels.append(y_2)

    
test_data = []
test_labels= []

for i in range(250, 300):
    x = random.randint(250, 292)
    y_1 = []
    y_2 = []
    for j in range(x, x+8):
        if j < x+7:
            y_1.append(foo[j])
        else:
            y_2.append(bar[j])
    test_data.append(y_1)
    test_labels.append(y_2)
    
train_data = np.array(train_data)
train_labels = np.array(train_labels)
test_data = np.array(test_data)
test_labels = np.array(test_labels)

In [7]:
print("Train info",train_data.shape, train_labels.shape)
print("Test info",test_data.shape, test_labels.shape)

Train info (250, 7) (250, 1)
Test info (50, 7) (50, 1)


In [8]:
#train_data = train_data.reshape((train_data.shape[0],28,28,1))
train_data = train_data.astype('float32')/2
#
#test_data = test_data.reshape((test_data.shape[0],28,28,1))
test_data = test_data.astype('float32')/2
print("test_labels",test_labels[:5])

train_labels_cat = keras.utils.to_categorical(train_labels)
test_labels_cat = keras.utils.to_categorical(test_labels)

test_labels [[1]
 [0]
 [1]
 [2]
 [2]]


In [9]:
print(train_labels_cat.shape)

(250, 3)


In [10]:
#print(test_labels)

In [11]:
from keras import models
from keras import layers

#
# Define the model
fcn_network = models.Sequential()
#
fcn_network.add(layers.Flatten(input_shape=[7]))
#
# Hidden
fcn_network.add(layers.Dense(100,activation='tanh'))
#
# Output
fcn_network.add(layers.Dense(3,activation='softmax'))
#
# Compile
fcn_network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# 
# Fit/save/print summary
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
fcn_results = fcn_network.fit(train_data,train_labels_cat,
                              epochs=10,
                              batch_size=128,
                              callbacks=callbacks, # Early stopping
                              validation_data=(test_data,test_labels_cat))
fcn_network.save('fully_trained_model_fcn_1.h5')

2022-11-29 13:01:35.855481: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-29 13:01:35.855582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p0063.ten.osc.edu): /proc/driver/nvidia/version does not exist
2022-11-29 13:01:35.856958: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10
2/2 [==============================] - 1s 252ms/step - loss: 1.1113 - accuracy: 0.3880 - val_loss: 1.0921 - val_accuracy: 0.4000
Epoch 2/10
2/2 [==============================] - 0s 27ms/step - loss: 1.0961 - accuracy: 0.3960 - val_loss: 1.0995 - val_accuracy: 0.4000
Epoch 3/10
2/2 [==============================] - 0s 49ms/step - loss: 1.0841 - accuracy: 0.4000 - val_loss: 1.1101 - val_accuracy: 0.4400
Epoch 4/10
2/2 [==============================] - 0s 35ms/step - loss: 1.0747 - accuracy: 0.4040 - val_loss: 1.1214 - val_accuracy: 0.4200
Epoch 5/10
2/2 [==============================] - 0s 31ms/step - loss: 1.0689 - accuracy: 0.4120 - val_loss: 1.1338 - val_accuracy: 0.4600
Epoch 6/10
2/2 [==============================] - 0s 40ms/step - loss: 1.0650 - accuracy: 0.4200 - val_loss: 1.1455 - val_accuracy: 0.3000
Epoch 7/10
2/2 [==============================] - 0s 34ms/step - loss: 1.0625 - accuracy: 0.4360 - val_loss: 1.1559 - val_accuracy: 0.2000
Epoch 8/10
2/2 [==========

In [12]:
# Define the model
fcn_network = models.Sequential()
#
fcn_network.add(layers.Flatten(input_shape=[7]))
#
# Hidden
fcn_network.add(layers.Dense(100,activation='tanh'))
fcn_network.add(layers.Dense(100,activation='tanh'))
fcn_network.add(layers.Dense(100,activation='tanh'))
#
# Output
fcn_network.add(layers.Dense(3,activation='softmax'))
#
# Compile
fcn_network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# 
# Fit/save/print summary
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
fcn_results = fcn_network.fit(train_data,train_labels_cat,
                              epochs=10,
                              batch_size=128,
                              callbacks=callbacks, # Early stopping
                              validation_data=(test_data,test_labels_cat))
fcn_network.save('fully_trained_model_fcn_2.h5')

Epoch 1/10
2/2 [==============================] - 1s 172ms/step - loss: 1.1339 - accuracy: 0.2960 - val_loss: 1.0577 - val_accuracy: 0.3400
Epoch 2/10
2/2 [==============================] - 0s 49ms/step - loss: 1.0813 - accuracy: 0.3720 - val_loss: 1.0690 - val_accuracy: 0.5200
Epoch 3/10
2/2 [==============================] - 0s 39ms/step - loss: 1.0701 - accuracy: 0.4120 - val_loss: 1.1020 - val_accuracy: 0.3800
Epoch 4/10
2/2 [==============================] - 0s 34ms/step - loss: 1.0583 - accuracy: 0.4400 - val_loss: 1.1308 - val_accuracy: 0.4800
Epoch 5/10
2/2 [==============================] - 0s 28ms/step - loss: 1.0461 - accuracy: 0.4600 - val_loss: 1.1605 - val_accuracy: 0.3000
Epoch 6/10
2/2 [==============================] - 0s 26ms/step - loss: 1.0385 - accuracy: 0.4840 - val_loss: 1.1908 - val_accuracy: 0.2200
Epoch 7/10
2/2 [==============================] - 0s 38ms/step - loss: 1.0378 - accuracy: 0.4720 - val_loss: 1.2189 - val_accuracy: 0.2200
Epoch 8/10
2/2 [==========

In [21]:
# Define the model
fcn_network = models.Sequential()
#
fcn_network.add(layers.Flatten(input_shape=[7]))
#
# Hidden
#fcn_network.add(layers.Dense(100,activation='tanh'))
fcn_network.add(layers.Dense(400,activation='tanh'))
#fcn_network.add(layers.Dense(100,activation='tanh'))
#
# Output
fcn_network.add(layers.Dense(3,activation='softmax'))
#
# Compile
fcn_network.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
# 
# Fit/save/print summary
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
fcn_results = fcn_network.fit(train_data,train_labels_cat,
                              epochs=10,
                              batch_size=128,
                              callbacks=callbacks, # Early stopping
                              validation_data=(test_data,test_labels_cat))
fcn_network.save('fully_trained_model_fcn_3.h5')

Epoch 1/10
2/2 [==============================] - 1s 134ms/step - loss: 1.1071 - accuracy: 0.3480 - val_loss: 1.1109 - val_accuracy: 0.4800
Epoch 2/10
2/2 [==============================] - 0s 22ms/step - loss: 1.0803 - accuracy: 0.3600 - val_loss: 1.1299 - val_accuracy: 0.1400
Epoch 3/10
2/2 [==============================] - 0s 23ms/step - loss: 1.0694 - accuracy: 0.3680 - val_loss: 1.1531 - val_accuracy: 0.0800
Epoch 4/10
2/2 [==============================] - 0s 22ms/step - loss: 1.0654 - accuracy: 0.3640 - val_loss: 1.1723 - val_accuracy: 0.1600
Epoch 5/10
2/2 [==============================] - 0s 40ms/step - loss: 1.0628 - accuracy: 0.3680 - val_loss: 1.1835 - val_accuracy: 0.1600
Epoch 6/10
2/2 [==============================] - 0s 31ms/step - loss: 1.0582 - accuracy: 0.3720 - val_loss: 1.1848 - val_accuracy: 0.2200
Epoch 7/10
2/2 [==============================] - 0s 42ms/step - loss: 1.0534 - accuracy: 0.3960 - val_loss: 1.1791 - val_accuracy: 0.2200
Epoch 8/10
2/2 [==========

In [22]:
from keras.models import load_model
test_list = [[1,2,0,2,0,1,1]]
test_list = np.array(test_list)

network_name = 'fully_trained_model_fcn_3.h5'
fcn_network = load_model(network_name)

print(test_list.shape)

(1, 7)


In [23]:
predictions = fcn_network.predict(test_list)

1/1 [==============================] - 0s 68ms/step


In [24]:
print(predictions)

[[0.40883783 0.20606153 0.3851006 ]]


In [25]:
print(np.argmax(predictions))

0
